In [1]:
import tensorflow as tf
import keras
import os
import core_values as cova

2025-09-14 22:40:19.476602: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

try:
    for i in gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

except Exception as e:
    print(f"{e}")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
path = "/mnt/d/Tojo Sir - Project/"
processed_path = os.path.join(path, "processed_data")

In [5]:
try:
    train_ds = tf.data.Dataset.load(os.path.join(processed_path, "train"))
    valid_ds = tf.data.Dataset.load(os.path.join(processed_path, "valid"))

except Exception as e:
    print(f"{e}")

I0000 00:00:1757875224.644137   86190 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2246 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [6]:
train = train_ds.prefetch(buffer_size = AUTOTUNE)
valid = valid_ds.prefetch(buffer_size = AUTOTUNE)

In [7]:
data_augmentation = keras.models.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomZoom(0.1, 0.1),
    keras.layers.RandomTranslation(0.1, 0.1),
    keras.layers.RandomRotation(0.2)
], name = "data_augmentation")

In [8]:
model = keras.models.Sequential([
    keras.layers.Input((cova.IMAGE_SIZE[0], cova.IMAGE_SIZE[1], 3)),

    #data_augmentation,

    keras.layers.Conv2D(16, (3,3), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Conv2D(32, (3,3), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Conv2D(64, (3,3), padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(activation=keras.activations.leaky_relu),
    keras.layers.MaxPool2D((2,2)),

    keras.layers.Flatten(),

    keras.layers.Dense(1024, activation = keras.activations.leaky_relu),
    keras.layers.Dense(256, activation = keras.activations.leaky_relu),
    keras.layers.Dense(64, activation = keras.activations.leaky_relu),
    keras.layers.Dense(4, activation = keras.activations.softmax)
])

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate= 0.001),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

In [10]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.1,
    patience = 3,
    verbose = 1,
    min_lr = 0.00000001
)

In [ ]:
model.fit(train,  validation_data = valid, epochs = 256, callbacks=[early_stopping, reduce_lr])

Epoch 1/256


2025-09-14 22:40:30.380152: I external/local_xla/xla/service/service.cc:163] XLA service 0x7861f4008240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-14 22:40:30.380205: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-09-14 22:40:30.467670: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-14 22:40:31.197919: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300
2025-09-14 22:40:31.937477: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:546] Omitted potentially buggy algorithm eng14{k25=2} for conv (f32[32,16,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,224,224]{3,2,1,0}, f32[16,3,3,3]{3,2,1,0}, f32[16]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call

1860/1862 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.6539 - loss: 2.4868

2025-09-14 22:47:48.637250: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:546] Omitted potentially buggy algorithm eng14{k25=2} for conv (f32[4,16,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,3,224,224]{3,2,1,0}, f32[16,3,3,3]{3,2,1,0}, f32[16]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false,"reification_cost":[]}
2025-09-14 22:47:48.758693: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:546] Omitted potentially buggy algorithm eng14{k25=2} for conv (f32[4,32,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,16,112,112]{3,2,1,0}, f32[32,16,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="_

1862/1862 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - accuracy: 0.6539 - loss: 2.4855

2025-09-14 22:50:41.642812: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:546] Omitted potentially buggy algorithm eng14{k25=2} for conv (f32[20,16,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,3,224,224]{3,2,1,0}, f32[16,3,3,3]{3,2,1,0}, f32[16]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false,"reification_cost":[]}
2025-09-14 22:50:41.828858: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:546] Omitted potentially buggy algorithm eng14{k25=2} for conv (f32[20,32,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,16,112,112]{3,2,1,0}, f32[32,16,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_targe

1862/1862 ━━━━━━━━━━━━━━━━━━━━ 618s 325ms/step - accuracy: 0.6778 - loss: 1.2364 - val_accuracy: 0.6463 - val_loss: 0.8771 - learning_rate: 0.0010
Epoch 2/256
1862/1862 ━━━━━━━━━━━━━━━━━━━━ 568s 305ms/step - accuracy: 0.7284 - loss: 0.7128 - val_accuracy: 0.6023 - val_loss: 1.5555 - learning_rate: 0.0010
Epoch 3/256
1862/1862 ━━━━━━━━━━━━━━━━━━━━ 476s 255ms/step - accuracy: 0.7376 - loss: 0.6980 - val_accuracy: 0.7715 - val_loss: 0.6566 - learning_rate: 0.0010
Epoch 4/256
1862/1862 ━━━━━━━━━━━━━━━━━━━━ 409s 220ms/step - accuracy: 0.7552 - loss: 0.6575 - val_accuracy: 0.8050 - val_loss: 0.6467 - learning_rate: 0.0010
Epoch 5/256
1862/1862 ━━━━━━━━━━━━━━━━━━━━ 410s 220ms/step - accuracy: 0.7659 - loss: 0.6264 - val_accuracy: 0.8185 - val_loss: 0.5852 - learning_rate: 0.0010
Epoch 6/256
1862/1862 ━━━━━━━━━━━━━━━━━━━━ 409s 219ms/step - accuracy: 0.7845 - loss: 0.5750 - val_accuracy: 0.7458 - val_loss: 0.7053 - learning_rate: 0.0010
Epoch 7/256
1862/1862 ━━━━━━━━━━━━━━━━━━━━ 427s 229ms/step